In [ ]:
# JD Analysis Agent Testing

This notebook focuses on testing and refining the Job Description Analysis Agent, which evaluates resumes against job descriptions.


In [ ]:
## 1. Setup and Dependencies


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import json
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Initialize LLM
llm = ChatOpenAI(model="gpt-4-turbo-preview")


In [ ]:
## 2. Test Data

Let's create some test resumes and job descriptions to evaluate.


In [ ]:
# Example resume for a software engineer
test_resume_1 = {
    "full_name": "John Doe",
    "contact_details": {
        "email": "john@example.com",
        "phone": "123-456-7890",
        "linkedin": "linkedin.com/in/johndoe"
    },
    "education": [
        {
            "degree": "BS Computer Science",
            "institution": "MIT",
            "year": "2020"
        }
    ],
    "work_experience": [
        {
            "company": "Tech Corp",
            "role": "Senior Software Engineer",
            "duration": "2020-2024",
            "key_achievements": [
                "Led development of microservices architecture",
                "Reduced system latency by 40%",
                "Mentored junior developers"
            ]
        }
    ],
    "skills": [
        "Python",
        "JavaScript",
        "React",
        "Node.js",
        "AWS",
        "Docker"
    ]
}

# Example job description for a senior software engineer
test_jd_1 = {
    "title": "Senior Software Engineer",
    "required_skills": [
        "Python",
        "JavaScript",
        "Cloud platforms (AWS/GCP)",
        "Microservices architecture"
    ],
    "preferred_skills": [
        "React",
        "Node.js",
        "Docker",
        "Kubernetes"
    ],
    "experience": "5+ years in software development",
    "education": "BS in Computer Science or related field"
}


In [ ]:
## 3. JD Analysis Agent Implementation


In [ ]:
def analyze_jd_match(resume: dict, job_description: dict) -> dict:
    """Analyze how well a resume matches a job description."""
    
    prompt = f"""Analyze how well the following resume matches the job description.
    
    Resume:
    {json.dumps(resume, indent=2)}
    
    Job Description:
    {json.dumps(job_description, indent=2)}
    
    Please provide a detailed analysis in the following format:
    1. Overall Match Score (0-10)
    2. Required Skills Match Analysis
    3. Preferred Skills Match Analysis
    4. Experience Match Analysis
    5. Education Match Analysis
    6. Key Strengths
    7. Areas for Improvement
    
    Format your response as a JSON object with these keys.
    """
    
    messages = [HumanMessage(content=prompt)]
    response = llm.invoke(messages)
    
    try:
        # Try to parse the response as JSON
        analysis = json.loads(response.content)
    except json.JSONDecodeError:
        # If JSON parsing fails, return the raw response
        analysis = {"raw_response": response.content}
    
    return analysis


In [ ]:
## 4. Test the Agent


In [ ]:
# Test with our first example
analysis_result = analyze_jd_match(test_resume_1, test_jd_1)
print(json.dumps(analysis_result, indent=2))


In [ ]:
## 5. Test with Different Scenarios


In [ ]:
# Test with a less qualified candidate
test_resume_2 = {
    "full_name": "Jane Smith",
    "education": [
        {
            "degree": "BS Information Technology",
            "institution": "State University",
            "year": "2022"
        }
    ],
    "work_experience": [
        {
            "company": "StartUp Inc",
            "role": "Junior Developer",
            "duration": "2022-2024",
            "key_achievements": [
                "Developed frontend features",
                "Fixed bugs in production code"
            ]
        }
    ],
    "skills": [
        "JavaScript",
        "React",
        "HTML/CSS"
    ]
}

# Test with second example
analysis_result_2 = analyze_jd_match(test_resume_2, test_jd_1)
print(json.dumps(analysis_result_2, indent=2))


In [ ]:
## 6. Visualization of Results


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_comparison(results1, results2):
    """Plot a comparison of two analysis results."""
    plt.figure(figsize=(12, 6))
    
    scores = {
        'Candidate 1': results1.get('overall_match_score', 0),
        'Candidate 2': results2.get('overall_match_score', 0)
    }
    
    sns.barplot(x=list(scores.keys()), y=list(scores.values()))
    plt.title('JD Match Score Comparison')
    plt.ylim(0, 10)
    plt.show()

# Plot comparison
plot_comparison(analysis_result, analysis_result_2)


In [ ]:
## 7. Refine the Analysis

Based on the test results, we can refine our analysis approach.


In [ ]:
def get_detailed_skill_match(resume: dict, jd: dict) -> dict:
    """Get a detailed breakdown of skill matches."""
    resume_skills = set(resume.get('skills', []))
    required_skills = set(jd.get('required_skills', []))
    preferred_skills = set(jd.get('preferred_skills', []))
    
    required_matches = resume_skills.intersection(required_skills)
    preferred_matches = resume_skills.intersection(preferred_skills)
    
    return {
        'required_skills_matched': list(required_matches),
        'required_skills_missing': list(required_skills - resume_skills),
        'preferred_skills_matched': list(preferred_matches),
        'preferred_skills_missing': list(preferred_skills - resume_skills),
        'additional_skills': list(resume_skills - (required_skills | preferred_skills))
    }

# Test the detailed skill match analysis
skill_analysis_1 = get_detailed_skill_match(test_resume_1, test_jd_1)
print("Candidate 1 Skill Analysis:")
print(json.dumps(skill_analysis_1, indent=2))

skill_analysis_2 = get_detailed_skill_match(test_resume_2, test_jd_1)
print("\nCandidate 2 Skill Analysis:")
print(json.dumps(skill_analysis_2, indent=2))
